In [ ]:
%pylab inline
import scipy as sp
from numba import jit
import os

In [ ]:
## load the conceptors code
%run python/conceptors.py

In [ ]:
# get current working path
cwd = os.getcwd()
print( cwd )

# load the network that we have saved before
# net = loadObject( cwd + "/models/generator_1505238661.223747.pickled" )
# net = loadObject( cwd + "/models/generator_1505299791.345897.pickled" )
net = loadObject( cwd + "/models/generator_1505307036.51016.pickled" )


In [ ]:
state = createGeneratorState( net )

In [ ]:
state = iterateGenerator(net, state, np.array([0.0, 1.0]), 1.0, 1.0)
state['output'][0]


In [ ]:
## load the functions for receiving and sending osc
%run python/oscserver.py

exciteValue = 1.0
morphValues = np.array([0.0, 1.0, 0.0])
leakRate = 1.0

def onSpectral(ind,f):
    global exciteValue
    exciteValue = f
    return True

def onLeakrate(ind,f):
    global leakRate
    leakRate = f
    return True

def onMorph(ind,f):
    global morphValues
    morphValues[ind] = f
    return True

def onExit():
    print( "exiting" )
    global keepRunning
    keepRunning = False
    oscserver.free()
    return True

oscserver = makeOSCServer(57120, 57400, onExit)
oscserver.onMorph = onMorph
oscserver.onSpectral = onSpectral
oscserver.onLeakrate = onLeakrate

oscserver.start()

In [ ]:
import time

keepRunning = True

while keepRunning:
#     print( morphValues )
    # net 1
    state = iterateGenerator(net, state, morphValues, exciteValue, leakRate )
    
    oscserver.send_value( "/output", state['output'][0] )
    oscserver.send_array( "/x/PG", state['x'].flatten() )
    
    time.sleep(1 / 10.)
    

In [ ]:

# oscserver.free()